In [1]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np
import re
# nltk.download('punkt')
# nltk.download('stopwords')

In [2]:
df = pd.read_csv('train_inout.csv', sep = ';', thousands=' ', decimal=',')
df

,Код,Проведена,Дата,ХозОперация,Поступление,Списание,Контрагент,Заказ,Наша организация,Наш р/счет,ЗаказСНашейСтороны,Менеджер,Комментарий 1,Комментарий 2
0,15,ИСТИНА,22.01.2014,30,0.0,3000.0,649,1296,98,10,194,NaN,Плата за прием наличных средств по документу N...,NaN
1,14,ИСТИНА,22.01.2014,30,0.0,990.0,649,1296,98,10,194,9.0,Предоставление средств криптографической защит...,NaN
2,11,ИСТИНА,22.01.2014,30,0.0,150.0,649,1296,98,10,194,9.0,Плата за прием наличных средств по документу N...,NaN
3,7,ИСТИНА,22.01.2014,30,0.0,600.0,649,1296,98,10,194,9.0,"Изготовление и заверение копий документов,пред...",NaN
4,4,ИСТИНА,22.01.2014,30,0.0,500.0,649,1296,98,10,194,NaN,Комиссия банка за оформление карточки с образц...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646,5037,ИСТИНА,30.06.2015,30,0.0,16.0,649,1296,98,10,194,NaN,Банковский ордер № 400676 от 30.06.2015 Комисс...,NaN
1647,4999,ИСТИНА,30.06.2015,56,38340.0,0.0,474,946,98,10,194,NaN,Платежное поручение № 981 от 30.06.2015 ВОЗВРА...,NaN
1648,4964,ИСТИНА,30.06.2015,60,4416.0,0.0,271,540,98,10,194,NaN,Платежное поручение № 531 от 29.06.2015 (340):...,NaN
1649,5032,ИСТИНА,30.06.2015,49,0.0,16828.0,639,1276,98,10,194,NaN,Платежное поручение № 454 от 30.06.2015 Оплата...,NaN


In [3]:
df1 = df[df['Списание'] > 0][['ХозОперация', 'Комментарий 1']]
df1

,ХозОперация,Комментарий 1
0,30,Плата за прием наличных средств по документу N...
1,30,Предоставление средств криптографической защит...
2,30,Плата за прием наличных средств по документу N...
3,30,"Изготовление и заверение копий документов,пред..."
4,30,Комиссия банка за оформление карточки с образц...
...,...,...
1643,49,Платежное поручение № 455 от 30.06.2015 Оплата...
1644,30,Банковский ордер № 148535 от 30.06.2015 Комисс...
1645,30,Банковский ордер № 400668 от 30.06.2015 Комисс...
1646,30,Банковский ордер № 400676 от 30.06.2015 Комисс...


In [4]:
res = df1['Комментарий 1'].str.lower().apply(word_tokenize)

In [5]:
rus_stopwords = stopwords.words('russian')

In [6]:
res1 = res.apply(lambda x: [word for word in x if word not in rus_stopwords]) 

In [7]:
res.apply(len), res1.apply(len)

(0       22
 1       23
 2       30
 3       28
 4       21
         ..
 1643    13
 1644    37
 1645    51
 1646    48
 1649    18
 Name: Комментарий 1, Length: 1180, dtype: int64,
 0       18
 1       20
 2       25
 3       24
 4       17
         ..
 1643     9
 1644    30
 1645    44
 1646    41
 1649    14
 Name: Комментарий 1, Length: 1180, dtype: int64)

In [8]:
snowball = SnowballStemmer(language="russian")

In [9]:
res1 = res1.apply(lambda x: [snowball.stem(word) for word in x])

In [10]:
res1

0       [плат, при, наличн, средств, документ, n, 2, с...
1       [предоставлен, средств, криптографическ, защит...
2       [плат, при, наличн, средств, документ, n, 1, с...
3       [изготовлен, заверен, коп, документ, ,, предос...
4       [комисс, банк, оформлен, карточк, образц, подп...
                              ...                        
1643    [платежн, поручен, №, 455, 30.06.2015, оплат, ...
1644    [банковск, ордер, №, 148535, 30.06.2015, комис...
1645    [банковск, ордер, №, 400668, 30.06.2015, комис...
1646    [банковск, ордер, №, 400676, 30.06.2015, комис...
1649    [платежн, поручен, №, 454, 30.06.2015, оплат, ...
Name: Комментарий 1, Length: 1180, dtype: object

In [11]:
res1 = res1.apply(lambda x: [word for word in x if not re.search(r'\d+', word)])
res1

0       [плат, при, наличн, средств, документ, n, сумм...
1       [предоставлен, средств, криптографическ, защит...
2       [плат, при, наличн, средств, документ, n, сумм...
3       [изготовлен, заверен, коп, документ, ,, предос...
4       [комисс, банк, оформлен, карточк, образц, подп...
                              ...                        
1643                  [платежн, поручен, №, оплат, пакет]
1644    [банковск, ордер, №, комисс, пп/пт, кб, дбо, с...
1645    [банковск, ордер, №, комисс, пп/пт, кб, дбо, с...
1646    [банковск, ордер, №, комисс, пп/пт, сб, дбо, с...
1649    [платежн, поручен, №, оплат, товар, бытов, хим...
Name: Комментарий 1, Length: 1180, dtype: object

In [12]:
res1 = res1.apply(lambda x: [word for word in x if not word == '№'])
res1

0       [плат, при, наличн, средств, документ, n, сумм...
1       [предоставлен, средств, криптографическ, защит...
2       [плат, при, наличн, средств, документ, n, сумм...
3       [изготовлен, заверен, коп, документ, ,, предос...
4       [комисс, банк, оформлен, карточк, образц, подп...
                              ...                        
1643                     [платежн, поручен, оплат, пакет]
1644    [банковск, ордер, комисс, пп/пт, кб, дбо, согл...
1645    [банковск, ордер, комисс, пп/пт, кб, дбо, согл...
1646    [банковск, ордер, комисс, пп/пт, сб, дбо, согл...
1649    [платежн, поручен, оплат, товар, бытов, хим, с...
Name: Комментарий 1, Length: 1180, dtype: object

In [13]:
res1 = res1.apply(lambda x: [word for word in x if not word == 'n'])
res1

0       [плат, при, наличн, средств, документ, сумм, ,...
1       [предоставлен, средств, криптографическ, защит...
2       [плат, при, наличн, средств, документ, сумм, (...
3       [изготовлен, заверен, коп, документ, ,, предос...
4       [комисс, банк, оформлен, карточк, образц, подп...
                              ...                        
1643                     [платежн, поручен, оплат, пакет]
1644    [банковск, ордер, комисс, пп/пт, кб, дбо, согл...
1645    [банковск, ордер, комисс, пп/пт, кб, дбо, согл...
1646    [банковск, ордер, комисс, пп/пт, сб, дбо, согл...
1649    [платежн, поручен, оплат, товар, бытов, хим, с...
Name: Комментарий 1, Length: 1180, dtype: object

In [14]:
vectorizer = CountVectorizer(min_df = 5)

In [15]:
df1['comm1'] = res1.str.join(' ')

In [16]:
X_train_counts = vectorizer.fit_transform(df1['comm1'])

In [17]:
vectorizer.vocabulary_

{'плат': 77,
 'средств': 98,
 'документ': 28,
 'сумм': 103,
 'тариф': 106,
 'ндс': 58,
 'облага': 62,
 'электрон': 123,
 'ключ': 45,
 'изготовлен': 34,
 'открыт': 71,
 'банковск': 8,
 'счет': 105,
 'руб': 91,
 'комисс': 46,
 'перечислен': 73,
 'валют': 14,
 'рф': 92,
 'согласн': 96,
 'действ': 24,
 'договор': 27,
 'аренд': 5,
 'оплат': 67,
 'феврал': 114,
 'лиценз': 50,
 'тарифн': 107,
 'план': 76,
 'базис': 7,
 'облагается': 63,
 'мебел': 55,
 'канцтовар': 42,
 'услуг': 112,
 'ндфл': 59,
 'январ': 126,
 'страхов': 99,
 'взнос': 17,
 'опс': 68,
 'зачисля': 32,
 'пфр': 87,
 'выплат': 20,
 'рег': 88,
 'осс': 70,
 'нс_пз': 60,
 'омс': 65,
 'ффомс': 117,
 'обязательн': 64,
 'социальн': 97,
 'страхован': 100,
 'бюджет': 13,
 'фсс': 116,
 'росс': 90,
 'участ': 113,
 'эл': 122,
 'аукцион': 6,
 'идентификатор': 33,
 'числ': 120,
 'обеспечен': 61,
 'форм': 115,
 'согл': 95,
 'исполнен': 36,
 'государствен': 22,
 'контракт': 47,
 'тнп': 108,
 'пп': 84,
 'сб': 93,
 'дбо': 23,
 'рко': 89,
 'rur': 

In [18]:
X_train_counts.shape

(1180, 127)

In [19]:
clf = MultinomialNB()
clf.fit(X_train_counts, df1['ХозОперация'])


MultinomialNB()

In [20]:
clf.predict(X_train_counts[0:2])

array([62, 34])